# Static Energy Budget

This very first model computes the earth surface temperature at the stationary point where the incoming energy from the sun is the same as the energy that is radiated out by the earth.

In [4]:
# some imports for later
from operator import add
from functools import partial

## Naked Earth Model

We start by looking at the energy that is hitting earth's surface from the sun. We need two values for earth:

1. The solar constant $C \approx 1350 \frac{W}{m^2}$, the incoming energy flow per unit area at the earth's distance from the sun
2. The degree of reflection, the *albedo*, which for the earch is roughly $alpha \approx 0.3$

The solar constant is actually a function of the planet's distance $D$ from the sun in Astronmical Units ($AU$), where 1 is the distance sun-earth: $L = \frac{C}{D^2}$

In [5]:
C = 1350
L = lambda dist: C / dist**2
alpha = 0.3

Only half of the earth is exposed to the sun. From the perspective of the parallel rays of sunlight, earth looks like a circle. So we can determine the amount of energy from the sun via a circle with the radius of the earth:

$L \cdot (1 - alpha) \cdot \pi \cdot R^2$

This incoming energy will increase the temperature of the earth. Let's assume for the moment that the entire planet will be heated up equally and instantly. Once the earth is warming, it starts emitting electromagnetic radiation itself according to the Stefan-Boltzman law: $E_{in} = \epsilon \sigma T^4$.

Here, $\epsilon$ describes the *emissivity*, which is 1 if a body emits energy across the whole spectrum. It is rougly 1 for the earth. And $\sigma$ is the Stefan-Boltzmann constant.

In [6]:
epsilon = 1
sigma = 5.67 * 10**-8 #[W/m^2/K^4]

Because we assumed that earth heats up homogeneously and instantly, it will emit radiation across it's whole surface, which can be approximated by the surface of a sphere:

$E_{out} = \epsilon \sigma T^4 \cdot 4 \pi R^2$

The sun will heat up the earth, and it in turn will emit enery out to space. If the temperature was too low, earth would absorb more engery and the temperature would rise, and similarly, it the temperature was too high, it would emit more energy than is coming in from the sun, and would cool down.

After some time, however, earth's temperature would converge to a stable state where incoming and outgoing energy flux would be the same:

$L \cdot (1 - alpha) \cdot \pi \cdot R^2 = \epsilon \sigma T^4 \cdot 4 \pi R^2$

We can simplify this equation, eliminate the actual radius of the earth, and determine the stable temperature as:

$T = (\frac{L \cdot (1 - alpha)}{4 \epsilon \sigma})^{\frac{1}{4}}$

In this formula, we have a few things that can vary, in particular the albedo of the earth and its distance to the sun.

In [7]:
def temp(albedo, dist=1):
    return (L(dist) * (1 - albedo) / (4 * epsilon * sigma))**(1/4)

With an albedo of 0.3 this gives us a surface temperature in Kelvin of:

In [8]:
t_surface = temp(albedo=0.3)
t_surface

254.06637407730736

Or, in Celcius:

In [9]:
Celsius = partial(add, -272.15)
Celsius(t_surface)

-18.08362592269262

This cannot the actual surface temperature, of course, because we haven't accounted for the earth's atmosphere at this point. But we have a model where we can hypothetically change the distance from the sun and the surface color of the earth.

## Introducing an Atmosphere

A good first abstraction for the atmosphere is a pane of glass around the earth. Electromagnetic radiation in the visible spectrum can penetrate this pane, but the infrared radiation emitted by the warm earth will interact with the atmosphere and heat it up, so it will now radiate itself. As the atmosphere is a pane, in other words: just a surface, it will radiate in two direction: out to space and back to earth. 

At equilibrium, the radiation emitted into space is still equal to the incoming flux from the sun. With $T_A$ being the temperature of the atmosphere, the equation is now:

$T_A = (\frac{L \cdot (1 - alpha)}{4 \epsilon \sigma})^{\frac{1}{4}}$

In [11]:
def temperature_atmosphere(albedo, dist=1):
    return (L(dist) * (1 - albedo) / (4 * epsilon * sigma))**(1/4)

In [12]:
t_atmosphere = temperature_atmosphere(0.3)
t_atmosphere

254.06637407730736

Additionally, we know that at equilibrium temperature, the energy leaving the earth surface ($\epsilon \sigma T_G^4$) is equal to the total energy emitted by the atmosphere:

$\epsilon \sigma T_G^4 \cdot 4 \pi R^2 = 2 \cdot \epsilon \sigma T_A^4 \cdot \pi R^2$

In other words, the earth's surface will be warmer than the atmosphere by a factor of:

$\frac{T_G}{T_A} = 2^{\frac{1}{4}}$

In [13]:
def temperature_surface(albedo, dist=1):
    return 2**(1/4) * temperature_atmosphere(albedo, dist)

In [14]:
t_surface = temperature_surface(0.3)
t_surface

302.1375397356768

Or, in degrees Celsius:

In [17]:
Celsius(t_surface)

29.98753973567682

This does not match the actual average surface temperature of the earth of 21.5°C perfectly, but it's not too bad as a first approximation.